# 千年讲堂的方形轮子 II

抛开Flag3不谈，本题是一道纯Web/Misc。Unicode is Magic!

### Flag 1 

> Flag 1：XTS模式除了最后两个分组，其他分组的加密结果只和该分组明文以及分组位置有关，和其他分组无关。于是你可以先生成多个ticket，再用它们拼凑成一个新ticket。注意JSON里面对象前后、字段间以及字段的key和value间的空格都是无关紧要的。

XTS 的每一个数据块是独立加密的，并不依赖于其它数据块。
AES-XTS 的分组大小是 16 字节 (128 位)，实际上我们可以任意构造数据块把一个payload给拼出来。先从题目源码里抽取一些用的上的部分，然后写点辅助函数方便我们解题：

In [34]:
import json
import time
import random

def split_blocks(data: bytes, block_size: int = 16) -> list[bytes]:
    return [data[i:i + block_size] for i in range(0, len(data), block_size)]

def gen_token():
    ALPHABET='qwertyuiopasdfghjklzxcvbnm1234567890'
    LENGTH=16
    return ''.join([random.choice(ALPHABET) for _ in range(LENGTH)])

def gen_data(l: int, name: str, stuid: str):
    if not 0<len(name)<=[99,22,18][l]:
        raise ValueError("姓名长度不正确")
    if not (len(stuid)==10 and stuid.isdigit()):
        raise ValueError("学号格式不正确")

    match l:
        case 0:        
            data = {
                'stuid': stuid,
                'name': name,
                'flag': False,
                'timestamp': int(time.time()),
            }
        case 1:        
            data = {
                'stuid': stuid,
                'name': name,
                'flag': False,
                'code': gen_token(),
                'timestamp': int(time.time()),
            }
        case 2:        
            data = {
                'stuid': stuid,
                'code': gen_token(),
                'name': name,
                'flag': False,
            }
        case _:
            raise ValueError("不支持的类型")

    _encoded = json.dumps(data).encode()
    return split_blocks(_encoded)

gen_data(0, "flag\": true", "1234567890")


[b'{"stuid": "12345',
 b'67890", "name": ',
 b'"flag\\": true", ',
 b'"flag": false, "',
 b'timestamp": 1761',
 b'207108}']

In [ ]:
import requests
from pprint import pprint
import re

URL = "https://prob14-s2evx8h2.geekgame.pku.edu.cn"

LEVEL = 1

def parse_ticket_info(html_content: str) -> dict:
    pattern = r"<b>(.*?)：</b>\s*(.*?)<"
    matches = re.findall(pattern, html_content)
    info_dict = {key.strip(): value.strip() for key, value in matches}
        
    return info_dict

def api_gen_ticket(LEVEL, name, stuid):
    response = requests.get(f"{URL}/{LEVEL}/gen-ticket", params={"name": name, "stuid": stuid})
    s = response.text
    pprint(s)
    p_idx = s.find("<br><p>")
    pe_idx = s.rfind("</p><br>")
    ticket = s[p_idx+7:pe_idx]
    return ticket

def api_query_ticket(LEVEL, ticket):
    print("querying ticket:", ticket)
    response = requests.get(f"{URL}/{LEVEL}/query-ticket", params={"ticket": ticket})
    pprint(response.text)
    return parse_ticket_info(response.text)

ticket = api_gen_ticket(LEVEL, "1234567", "1234567890")
print("ticket:", ticket)
result = api_query_ticket(LEVEL, ticket)
print("result:", result)

('<p>已为您生成购票凭证：</p><br><p>XXM5N+IP9SqiPOIkLC2IbaArJZKVzDd7YIXYzVvyIv8Zdbd1huRjKW9qqAsjBXtRT18rL/b6Co2JVCHMoyxvHj7oPI1R+hsnv5E/UZWkyCdG0A==</p><br><p><a '
 'href="/">返回</a></p>')
ticket: XXM5N+IP9SqiPOIkLC2IbaArJZKVzDd7YIXYzVvyIv8Zdbd1huRjKW9qqAsjBXtRT18rL/b6Co2JVCHMoyxvHj7oPI1R+hsnv5E/UZWkyCdG0A==
querying ticket: XXM5N+IP9SqiPOIkLC2IbaArJZKVzDd7YIXYzVvyIv8Zdbd1huRjKW9qqAsjBXtRT18rL/b6Co2JVCHMoyxvHj7oPI1R+hsnv5E/UZWkyCdG0A==
('<!doctype html>\n'
 '<html>\n'
 '<head>\n'
 '    <meta charset=utf-8>\n'
 '    <title>千年讲堂网上购票系统</title>\n'
 '</head>\n'
 '<body>\n'
 '    <p>解密得到您的购票信息如下</p>\n'
 '    <br>\n'
 '    <p><b>姓名：</b> 1234567</p>\n'
 '    <p><b>学号：</b> 1234567890</p>\n'
 '    <p><b>需要礼品：</b> False</p>\n'
 '    <p><b>礼品兑换码：</b> </p>\n'
 '    <p><b>时间戳：</b> 1761220533</p>\n'
 '    <br>\n'
 '    <p><a href="/">返回</a></p>\n'
 '</body>\n'
 '</html>')
result: {'姓名': '1234567', '学号': '1234567890', '需要礼品': 'False', '礼品兑换码': '', '时间戳': '1761220533'}


name的值显然是我们可以任意控制的。

注意到在JS序列化的时候，`"`等符号会被转义，所以如果要在字符串里构造这些符号必须保证上一行的最后一个字符是`\`。

Python的JS解析允许重复的Key，后面的值覆盖前面的。所以我们不仅要能产生`"flag": true"，还有把最后的flag换成别的名字。

一个可能的payload是：

```
3 | "123456", "flag
4 | ":true,      
5 | "abcdefefefefefe           
6 | flag": false,
```

In [ ]:
from pprint import pprint

# use line 1-3
# we get "123456", "flag from here on block 3
pprint(gen_data(0, "1234567", "1234567890"))

[b'{"stuid": "12345',
 b'67890", "name": ',
 b'"1234567", "flag',
 b'": false, "times',
 b'tamp": 176120876',
 b'6}']


In [88]:
# use line 4
# we get ": true,         from here on block 4
pprint(gen_data(0, "1234567890abcd\": true,        ", "1234567890"))

[b'{"stuid": "12345',
 b'67890", "name": ',
 b'"1234567890abcd\\',
 b'": true,        ',
 b'", "flag": false',
 b', "timestamp": 1',
 b'761208820}']


In [80]:
# use line 5
# we get "abcdefefefefefe on line 5
pprint(gen_data(0, "1234567890abcdefghijklmnopq123\"567890abcdefsds", "1234567890"))

[b'{"stuid": "12345',
 b'67890", "name": ',
 b'"1234567890abcde',
 b'fghijklmnopq123\\',
 b'"567890abcdefsds',
 b'", "flag": false',
 b', "timestamp": 1',
 b'761208662}']


In [89]:
# use line 6 and later
# we get flag": false, "t on line 6
pprint(gen_data(0, "1234567890abcdefghijklmnopq1234567890abcdef", "1234567890"))

[b'{"stuid": "12345',
 b'67890", "name": ',
 b'"1234567890abcde',
 b'fghijklmnopq1234',
 b'567890abcdef", "',
 b'flag": false, "t',
 b'imestamp": 17612',
 b'08844}']


In [ ]:
# JSON格式正确！
json.loads(b''.join([
 b'{"stuid": "12345',
 b'67890", "name": ',
 b'"1234567", "flag',

 b'": true,        ',

 b'"567890abcdefsds',

 b'flag": false, "t',
 b'imestamp": 17612',
 b'08844}']))

{'stuid': '1234567890',
 'name': '1234567',
 'flag': True,
 '567890abcdefsdsflag': False,
 'timestamp': 1761208844}

In [139]:
import base64

def _dec_blocks(level, name, stuid):
    return split_blocks(base64.b64decode(api_gen_ticket(level, name, stuid)))

def _enc_blocks(level, blocks):
    payload = b''.join(blocks)
    return api_query_ticket(level, base64.b64encode(payload).decode())

s123 = _dec_blocks(1, "1234567", "1234567890")
s4 = _dec_blocks(1, "1234567890abcd\": true,        ", "1234567890")
s5 = _dec_blocks(1, "1234567890abcdefghijklmnopq123\"567890abcdefsds", "1234567890")
s678 = _dec_blocks(1, "1234567890abcdefghijklmnopq1234567890abcdef", "1234567890")

sN = s123[:3] + s4[3:4] + s5[4:5] + s678[5:]
_enc_blocks(1, sN)

querying ticket: XXM5N+IP9SqiPOIkLC2IbaArJZKVzDd7YIXYzVvyIv8Zdbd1huRjKW9qqAsjBXtRcxXZLQAmiAlvuyPl2CoiGSNRIWtL+fYRIWWKLrdnhVVOVrpw0CmfREFO10KNJ4C8x/aL0cH8saHZOiBmglRZLgtOjvDK6g==
('<!doctype html>\n'
 '<html>\n'
 '<head>\n'
 '    <meta charset=utf-8>\n'
 '    <title>千年讲堂网上购票系统</title>\n'
 '</head>\n'
 '<body>\n'
 '    <p>解密得到您的购票信息如下</p>\n'
 '    <br>\n'
 '    <p><b>姓名：</b> 1234567</p>\n'
 '    <p><b>学号：</b> 1234567890</p>\n'
 '    <p><b>需要礼品：</b> True</p>\n'
 '    <p><b>礼品兑换码：</b> </p>\n'
 '    <p><b>时间戳：</b> 1761211032</p>\n'
 '    <br>\n'
 '    <p><a href="/">返回</a></p>\n'
 '</body>\n'
 '</html>')


{'姓名': '1234567',
 '学号': '1234567890',
 '需要礼品': 'True',
 '礼品兑换码': '',
 '时间戳': '1761211032'}

### Flag 2

> 拼凑密文不仅可以伪造ticket，还可以把code泄露出来。此题用户名限制的是字符数量，一个字符经过JSON encode后可能变成多个字节。

Flag2多了一个code字段，而且正好16个字符，仅仅注入name做不到啊！

> 在Python里面运行{i: len(json.dumps(chr(i))) - 2 for i in range(0x30000) if chr(i).isdigit()}，看看结果是什么。

哦，还有这回事？真神奇啊Unicode。

In [ ]:
# 牛逼兄弟
{i: len(json.dumps(chr(i))) - 2 for i in range(0x30000) if chr(i).isdigit()}

{48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 178: 6,
 179: 6,
 185: 6,
 1632: 6,
 1633: 6,
 1634: 6,
 1635: 6,
 1636: 6,
 1637: 6,
 1638: 6,
 1639: 6,
 1640: 6,
 1641: 6,
 1776: 6,
 1777: 6,
 1778: 6,
 1779: 6,
 1780: 6,
 1781: 6,
 1782: 6,
 1783: 6,
 1784: 6,
 1785: 6,
 1984: 6,
 1985: 6,
 1986: 6,
 1987: 6,
 1988: 6,
 1989: 6,
 1990: 6,
 1991: 6,
 1992: 6,
 1993: 6,
 2406: 6,
 2407: 6,
 2408: 6,
 2409: 6,
 2410: 6,
 2411: 6,
 2412: 6,
...
 123201: 12,
 123202: 12,
 123203: 12,
 123204: 12,
 123205: 12,
 123206: 12,
 123207: 12,
 123208: 12,
 123209: 12,
 123632: 12,
 123633: 12,
 123634: 12,
 123635: 12,
 123636: 12,
 123637: 12,
 123638: 12,
 123639: 12,
 123640: 12,
 123641: 12,
 125264: 12,
 125265: 12,
 125266: 12,
 125267: 12,
 125268: 12,
 125269: 12,
 125270: 12,
 125271: 12,
 125272: 12,
 125273: 12,
 127232: 12,
 127233: 12,
 127234: 12,
 127235: 12,
 127236: 12,
 127237: 12,
 127238: 12,
 127239: 12,
 127240: 12,
 127241: 12,
 127242: 12,

先看看正常生成的签名长什么样：

In [145]:
t = api_gen_ticket(2, '123', '2342342342')
api_query_ticket(2, t)

querying ticket: KmiI/nLUEt2EmqT5CU2+ptHIn41qcBHXlj759IpC6V6jsRaLHMKqaDAHErtKzlpj+vuKePL7Hbu0FAL6I5VWHdRtOUz+jnPtdwhA2V6/sD3Jcc+h5YGBOHd2AmyHaKio1enXueETh3o0Mg==
('<!doctype html>\n'
 '<html>\n'
 '<head>\n'
 '    <meta charset=utf-8>\n'
 '    <title>千年讲堂网上购票系统</title>\n'
 '</head>\n'
 '<body>\n'
 '    <p>解密得到您的购票信息如下</p>\n'
 '    <br>\n'
 '    <p><b>姓名：</b> 123</p>\n'
 '    <p><b>学号：</b> 2342342342</p>\n'
 '    <p><b>需要礼品：</b> False</p>\n'
 '    <p><b>礼品兑换码：</b> a65t************</p>\n'
 '    <p><b>时间戳：</b> 1761211449</p>\n'
 '    <br>\n'
 '    <p><a href="/">返回</a></p>\n'
 '</body>\n'
 '</html>')


{'姓名': '123',
 '学号': '2342342342',
 '需要礼品': 'False',
 '礼品兑换码': 'a65t************',
 '时间戳': '1761211449'}

In [205]:
pprint(gen_data(1, "1234567", "1"*10))

[b'{"stuid": "11111',
 b'11111", "name": ',
 b'"1234567", "flag',
 b'": false, "code"',
 b': "ilb92pon9ohtw',
 b'0z3", "timestamp',
 b'": 1761213345}']


从上述输出可以看到，code没法正常回显，而我们最后传的payload里的code必须和回显的code一致，需要想办法把name或者stuid设置成code。也就是说，我们必须能想办法把code泄露到单独的一行，并且还需要一个`"name": "`在行尾的Payload。这两个任务其实可以通过构造stuid一次性完成：

len(chr(66720)*a + chr(6474) * b + "1"*c) = N * 16 + 8, a + b + c = 9

0, 3, 7  N = 1

1, 4, 5  N = 2

2, 5, 3  N = 3

3, 6, 1  N = 4

In [ ]:
# leak code
# XXX", "name": "
# code on line 7
pprint(gen_data(1, "123456", chr(66720)*3 + chr(6474) * 6 + "1"*1))

[b'{"stuid": "\\ud80',
 b'1\\udca0\\ud801\\ud',
 b'ca0\\ud801\\udca0\\',
 b'u194a\\u194a\\u194',
 b'a\\u194a\\u194a\\u1',
 b'94a1", "name": "',
 b'123456", "flag":',
 b' false, "code": ',
 b'"7z9eairta1oboyw',
 b'r", "timestamp":',
 b' 1761215909}']


Flag 2 说是限制了输入的长度，但是在Unicode的帮助下这都不是事：

In [ ]:
# unicode ! 
len("🏄‍♂️🌊❄️🦅".encode())

27

In [245]:
# here we get a code line on line 7
pprint(gen_data(1, "🏄‍♂️" + "1" * 6, "1234567890"))

[b'{"stuid": "12345',
 b'67890", "name": ',
 b'"\\ud83c\\udfc4\\u2',
 b'00d\\u2642\\ufe0f1',
 b'11111", "flag": ',
 b'false, "code": "',
 b'1jsj11e7gtgc37zv',
 b'", "timestamp": ',
 b'1761216244}']


In [348]:
s_codeprefix = _dec_blocks(2, "123456", chr(66720)*3 + chr(6474) * 6 + "1"*1)
s_code = _dec_blocks(2, "🏄‍♂️" + "1" * 6, "1234567890")
s_getcode = s_codeprefix[:6] + s_code[6:]
code_parsed = _enc_blocks(2, s_getcode)
print(code_parsed['姓名'])

('<p>已为您生成购票凭证：</p><br><p>mifMZ2p7NHCbj9guhm3n0CxxMESRkVo0PO+U+3NWfSsMjVF47pdSy8LWG+fUffA2/EmYvJ2tU8iU/+b+AELKCqLnZUA8KdqbM2AFtmd/68QblkQrrLx0LoITzoutDaZVtJbDF+/0LXQpyDoLg547GhAuwqiFy87/+KXCdDNDzc5t4ZLHBaf005BjeX6J7V9dI1ieFQnYPp8zv3CbPSzC+3hetxa2iHPWEtSxSw==</p><br><p><a '
 'href="/">返回</a></p>')
('<p>已为您生成购票凭证：</p><br><p>nqwnvMWhV6vxWdPNzqsUDtep2B/0lF2OJ0GedfOM1KXxOKysIcmNzmTd8NoQIeV5p+dlDxfBbjGCepqQlZK79U71MCQRuAq5C84AO6MA5+LOHdu8SHeuDNypXxKgoiMLn8818EzWQDzbg5f59ONET2cRGU6ZDKqJ+P58VOo9E53lamIDWXnJHVKw8g==</p><br><p><a '
 'href="/">返回</a></p>')
querying ticket: mifMZ2p7NHCbj9guhm3n0CxxMESRkVo0PO+U+3NWfSsMjVF47pdSy8LWG+fUffA2/EmYvJ2tU8iU/+b+AELKCqLnZUA8KdqbM2AFtmd/68QblkQrrLx0LoITzoutDaZVn8818EzWQDzbg5f59ONET2cRGU6ZDKqJ+P58VOo9E53lamIDWXnJHVKw8g==
('<!doctype html>\n'
 '<html>\n'
 '<head>\n'
 '    <meta charset=utf-8>\n'
 '    <title>千年讲堂网上购票系统</title>\n'
 '</head>\n'
 '<body>\n'
 '    <p>解密得到您的购票信息如下</p>\n'
 '    <br>\n'
 '    <p><b>姓名：</b> a2jksobiutp0r6m6</p>\n'
 '    <p><b>学号：</b> 

很好，我们现在拿到了一组明文的Code以及它对应的密文，接下来思考一下如何让密文行不变的情况下把flag的值设置为true。

第3行需要把"flag"控制在一个恰好的位置，让我们能够在第4行通过name设置`"flag":true`。比较头疼的是如何让后面的`"flag":false无效。经过几轮迭代，我想到了以下办法：
 
```python
 b'{"stuid": "11111', # <- s_3
 b'11111", "name": ', # <- s_3
 b'"1111111", "flag', # <- s_3
 b'"     :true,", "', # <- s_4
 b': false, "code":', # <- s_5
 b'false, "code": "', # <- s_code
 b'3shyynbrxqmjnus5', # <- s_code
 b'", "timestamp": ', # <- s_code
 b'1761217224}' # <- s_code
```

In [351]:
payload_3 = gen_data(1, "1111111", "1"*10)
pprint(payload_3)
s_3 = _dec_blocks(2, "1111111", "1"*10)

[b'{"stuid": "11111',
 b'11111", "name": ',
 b'"1111111", "flag',
 b'": false, "code"',
 b': "z0zyjspyf3k9i',
 b'wnx", "timestamp',
 b'": 1761220547}']
('<p>已为您生成购票凭证：</p><br><p>GTJdUXcNrfrclt164gMrNjoKhZY6kWneF+EUoVK0yhr2TPQXkKJrJI47zmD2yIN39VbWJQlZzKSrI5j1R3CvQAvdqM85KOAWU6H1VVz5WfKd2Et0dXTOj43ZaoQwGSrVJo13/2iceL/LqQJnQOQ=</p><br><p><a '
 'href="/">返回</a></p>')


In [352]:
_name = "冲浪  \"     :true,"
_stuid = "1"*10
payload_4 = gen_data(1, _name, _stuid)
s_4 = _dec_blocks(2, _name, _stuid)
pprint(payload_4)

('<p>已为您生成购票凭证：</p><br><p>GTJdUXcNrfrclt164gMrNjoKhZY6kWneF+EUoVK0yhpUqY4MOA7G9l/Jfgt9tOIY4+Z6dVhd0Y/e0iM8gS2gmzw3dDgH1GjeE/riesUIEuA/QqiQtnCqXAj2pjevkRtaZpA+dwBoMlI5iD6XCzDptgzDbEzgw5a7O4PvDapmCoODmQ==</p><br><p><a '
 'href="/">返回</a></p>')
[b'{"stuid": "11111',
 b'11111", "name": ',
 b'"\\u51b2\\u6d6a  \\',
 b'"     :true,", "',
 b'flag": false, "c',
 b'ode": "0sryhj289',
 b'itovxfz", "times',
 b'tamp": 176122054',
 b'7}']


In [353]:
_name = "莉莉子1111"
_stuid = "1"*10
payload_5 = gen_data(1, _name, _stuid)
s_5 = _dec_blocks(2, _name, _stuid)
pprint(payload_5)

('<p>已为您生成购票凭证：</p><br><p>GTJdUXcNrfrclt164gMrNjoKhZY6kWneF+EUoVK0yhohuG30HphMz4fv7VJ/lig/rw+rxXvgGR2JqdmFAfx/uLh7MhGkjXWA39KGkpY9GhOddXZdiTY4M8YWZvEMfHRvY4V/snY+QECuSLGQRzUBjmx6PXQKSFVFeHjNdr8=</p><br><p><a '
 'href="/">返回</a></p>')
[b'{"stuid": "11111',
 b'11111", "name": ',
 b'"\\u8389\\u8389\\u5',
 b'b501111", "flag"',
 b': false, "code":',
 b' "dvestcmfe402b2',
 b'nu", "timestamp"',
 b': 1761220547}']


In [ ]:
# okay we are still able to manipulate first 6 lines
json.loads(b''.join([
 b'{"stuid": "11111', # <- s_3
 b'11111", "name": ', # <- s_3
 b'"1111111", "flag', # <- s_3
 b'"     :true,", "', # <- s_4
 b': false, "code":', # <- s_5
 b'false, "code": "', # <- s_code
 b'3shyynbrxqmjnus5', # <- s_code
 b'", "timestamp": ', # <- s_code
 b'1761217224}' # <- s_code
]))

{'stuid': '1111111111',
 'name': '1111111',
 'flag': True,
 ', ': False,
 'code': '3shyynbrxqmjnus5',
 'timestamp': 1761217224}

In [ ]:
s_payload = s_3[:3] + s_4[3:4] + s_5[4:5] + s_code[5:]
_enc_blocks(2, s_payload)

querying ticket: GTJdUXcNrfrclt164gMrNjoKhZY6kWneF+EUoVK0yhr2TPQXkKJrJI47zmD2yIN34+Z6dVhd0Y/e0iM8gS2gm7h7MhGkjXWA39KGkpY9GhPOHdu8SHeuDNypXxKgoiMLn8818EzWQDzbg5f59ONET2cRGU6ZDKqJ+P58VOo9E53lamIDWXnJHVKw8g==
('<!doctype html>\n'
 '<html>\n'
 '<head>\n'
 '    <meta charset=utf-8>\n'
 '    <title>千年讲堂网上购票系统</title>\n'
 '</head>\n'
 '<body>\n'
 '    <p>解密得到您的购票信息如下</p>\n'
 '    <br>\n'
 '    <p><b>姓名：</b> 1111111</p>\n'
 '    <p><b>学号：</b> 1111111111</p>\n'
 '    <p><b>需要礼品：</b> True</p>\n'
 '    <p><b>礼品兑换码：</b> a2jk************</p>\n'
 '    <p><b>时间戳：</b> 1761220547</p>\n'
 '    <br>\n'
 '    <p><a href="/">返回</a></p>\n'
 '</body>\n'
 '</html>')


{'姓名': '1111111',
 '学号': '1111111111',
 '需要礼品': 'True',
 '礼品兑换码': 'a2jk************',
 '时间戳': '1761220547'}